[Pricing information](https://aws.amazon.com/cloudwatch/pricing/):

In [1]:
static readonly (int count, double price)[] tiersCosts = {(10_000, 0.3), (240_000, 0.1), (750_000, 0.05), (-1, 0.02)};

The below script needs to be able to find the current output cell; this is an easy method to get it.

Method to calculate prices:

In [2]:
static double CalculateCustomMetricsPrice(int metricsCount) =>
    tiersCosts.Aggregate((count: metricsCount, price: 0d), (total, tier) => {        
        var tierCount = tier.count > 0 ? Math.Min(tier.count, total.count) : total.count;
        return (total.count - tierCount, total.price += tierCount * tier.price);
    }).price;

## Modify this values to change graph:

In [8]:
const int NumberOfEndpoints = 115; // current endpoints count as of 04/28/2020
const int NumberOfCommonEndpoints = 30;
private static readonly int[] CommonEndpointIndexes = Enumerable.Range(0, NumberOfCommonEndpoints).ToArray(); 
const int NumberOfUniqueEnpoints = 15;
const int NumberOfCurrentOrgs = 202;
const int NumberOfAllOrgs = 1000;
const double SuccessfulRequestsPercentage = 99.99;
static int[] AverageNumberOfRequestsByOrgPerMonth = {1_000, 20_000, 40_000, 60_000, 100_000};

Graphing helpers:

In [13]:
// true - successful, false - failure
static Func<bool> GetRequestStatus(Random r, double successProbability) => () =>  (double)r.Next(0, 100) + r.NextDouble() <= successProbability;

private static readonly int[] CommonEndpointIndexes = Enumerable.Range(0, NumberOfCommonEndpoints).ToArray(); 
private static readonly Random Rnd = new Random((int)DateTime.Now.Ticks);
static int[] GetOrgEndpoints()
{
    if(NumberOfEndpoints < (NumberOfCommonEndpoints + NumberOfUniqueEnpoints)) throw new Exception($"{nameof(NumberOfEndpoints)} should be greater than {nameof(NumberOfCommonEndpoints)} + {nameof(NumberOfUniqueEnpoints)}");
    var result = new int[NumberOfCommonEndpoints + NumberOfUniqueEnpoints];
    CommonEndpointIndexes.CopyTo(result, 0);
    var unique = new HashSet<int>();
    while (unique.Count < NumberOfUniqueEnpoints)
    {
        unique.Add(Rnd.Next(NumberOfCommonEndpoints, NumberOfEndpoints));
    }
    unique.CopyTo(result, NumberOfCommonEndpoints);
    return result;
}

class Request
{
    public int Endpoint { get; set; }
    public int Org { get; set; }
    public int Status { get; set; } // 1 - success, 0 - failure

    protected bool Equals(Request other) => Endpoint == other.Endpoint && Org == other.Org && Status == other.Status;
    public override bool Equals(object obj) => Equals((Request) obj);
    public override int GetHashCode() => HashCode.Combine(Endpoint, Org, Status);
}

static IReadOnlyCollection<Request> MakeRequests(int orgId, int numberOfRequests)
{
    var enpoints = GetOrgEndpoints();
    var endpointIndex = 0;
    var getStatus = GetRequestStatus(Rnd, SuccessfulRequestsPercentage);

    var result = new HashSet<Request>();
    for (int i = 0; i < numberOfRequests; i++)
    {
        result.Add(new Request {Org = orgId, Endpoint = enpoints[endpointIndex], Status = getStatus() ? 1 : 0});
        endpointIndex = endpointIndex + 1 == enpoints.Length ? 0 : endpointIndex + 1;
    }

    return result;
}


In [5]:
using XPlot.Plotly;

## Cost of one Org

In [14]:
var singleOrgCostPerCustomMetric = new Graph.Bar()
{
    x = AverageNumberOfRequestsByOrgPerMonth,
    y = AverageNumberOfRequestsByOrgPerMonth.Select(requests => MakeRequests(1234, requests).Count),
    name = $"Cost"
};
var singleOrgChart = Chart.Plot(new []{ singleOrgCostPerCustomMetric });
singleOrgChart.WithLayout(new Layout.Layout
{
    title = "Cost of a single organization",
    xaxis = new Graph.Xaxis{ title = "Number of request per month"},
    yaxis = new Graph.Yaxis{ title = "Price, $"},

});
display(singleOrgChart);

## Cost of multiple orgs
*Might take half a minute of more to run the requests simulation, be patient*

In [15]:
var currentOrgs = Enumerable.Range(0, NumberOfCurrentOrgs).ToArray();
var allOrgs = Enumerable.Range(0, NumberOfAllOrgs).ToArray();

var currentOrgsCostPerCustomMetric = new Graph.Bar()
{
    x = AverageNumberOfRequestsByOrgPerMonth,
    y = AverageNumberOfRequestsByOrgPerMonth.Select(requests => currentOrgs.Select(orgId => MakeRequests(orgId, requests).Count).Sum()),
    name = $"Current number of orgs ({NumberOfCurrentOrgs})"
};

var allOrgsCostPerCustomMetric = new Graph.Bar()
{
    x = AverageNumberOfRequestsByOrgPerMonth,
    y = AverageNumberOfRequestsByOrgPerMonth.Select(requests => allOrgs.Select(orgId => MakeRequests(orgId, requests).Count).Sum()),
    name = $"All orgs ({NumberOfAllOrgs})"
};

var orgsChart = Chart.Plot(new []{ currentOrgsCostPerCustomMetric, allOrgsCostPerCustomMetric });
orgsChart.WithLayout(new Layout.Layout
{
    title = "Cost of custom metrics",
    xaxis = new Graph.Xaxis{ title = "Number of request per month"},
    yaxis = new Graph.Yaxis{ title = "Price, $"},

});
display(orgsChart);